In [2]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import constants as const
from astropy import units as u

c = const.c.value
G = const.G.value
M_sun = const.M_sun.value
M_earth = const.M_earth.value

In [ ]:
def rel_len(x_1, x_2):
    r_vec = x_1 - x_2
    r = np.linalg.norm(r_vec)
    return r

def fourdot(V_1, V_2):
    return V_1[0]*V_2[0] - V_1[1]*V_2[1] - V_1[2]*V_2[2]

def rel_len(x_1, x_2):
    r_vec = x_1 - x_2
    r = np.linalg.norm(r_vec)
    return r

def fourdot(V_1, V_2):
    return V_1[0]*V_2[0] - V_1[1]*V_2[1] - V_1[2]*V_2[2]

def findR_SN(m_1, m_2, DL):
    if DL == True:
        c = 1
        G = 1
    else:
        c = const.c.value
        G = const.G.value
    R_S = 2 * (m_1 + m_2) * G/c**2
    return R_S

def findR_S(P_1, P_2, DL):
    if DL == True:
        c = 1
        G = 1
    else:
        c = const.c.value
        G = const.G.value
    R_S = 2 * (P_1[0]/c + P_2[0]/c) * G/c**2
    return R_S

def findrCM(x_1, x_2, m_1, m_2):
    M = m_1 + m_2
    r_cm = (m_1*x_1 + m_2*x_2)/M
    return r_cm

def γ(v, DL):
    if DL == True:
        c = 1
    else:
        c = const.c.value
    return 1/np.sqrt(1 - np.dot(v,v)/c**2)

def three_p(v, m, DL):
    return γ(v, DL)*m*v

def four_P(p, m, DL):
    if DL == True:
        c = 1
    else:
        c = const.c.value
    return np.array([np.sqrt(np.dot(p, p)*c**2 + m**2*c**4)/c, p[0], p[1]])

def rotate(V, θ):
    rot = np.array([[np.cos(θ), -np.sin(θ)], [np.sin(θ), np.cos(θ)]])
    return np.dot(rot, V)

In [17]:
def accelerationN(x_1, x_2, m_1, m_2, DL):
    if DL == True:
        G = 1
    else:
        G = const.G.value
    
    r_vec = x_1 - x_2
    r = rel_len(x_1, x_2)
    n = r_vec/r

    a_1 = - G*m_2*n/r**2
    a_2 =   G*m_1*n/r**2
    return np.array([a_1, a_2])
    
def accelerationPM(x_1, x_2, p_1, p_2, m_1, m_2, DL):
    if DL == True:
        G, c = 1, 1
    else:
        c = const.c.value
        G = const.G.value
    
    r = rel_len(x_1, x_2)
    r_vec = x_1 - x_2


    E_1 = np.sqrt((m_1*c**2)**2 + (p_1*c)**2)
    E_2 = np.sqrt((m_2*c**2)**2 + (p_2*c)**2)
    c_1 = -2*(1 + np.dot(p_1, p_2)/(m_1*m_2))**2 + (1 + np.dot(v_1, v_1))*(1 + np.dot(v_2, v_2))
    dc_1dp_1 = -4*p_2/(m_1*m_2)*(1 + np.dot(p_1, p_2)/(m_1*m_2)) + 2*p_1/m_1**2*(1 + np.dot(p_2, p_2)/m_2**2)
    dc_1dp_2 = -4*p_1/(m_1*m_2)*(1 + np.dot(p_1, p_2)/(m_1*m_2)) + 2*p_2/m_2**2*(1 + np.dot(p_1, p_1)/m_1**2)
    # ξ = E_1*E_2/(E_1 + E_2)**2

    p_dot_1 =  G*m_1*m_2*c_1*r_vec/(E_1*E_2*r**3)
    p_dot_2 = -G*m_1*m_2*c_1*r_vec/(E_1*E_2*r**3)
    c_1_dot = -4*(1 + np.dot(p_1, p_2)/(m_1*m_2))*(np.dot(p_dot_1, p_2) + np.dot(p_1, p_dot_2))/(m_1*m_2) + 2*( np.dot(p_1, p_dot_1)/m_1**2*(1 + np.dot(p_2, p_2)/m_2**2) + np.dot(p_2, p_dot_2)/m_2**2*(1 + np.dot(p_1, p_2)/m_1**2) )
    dc_1dp_1_dot = -4*p_dot_2/(m_1*m_2)*(1 + np.dot(p_1, p_2)/(m_1*m_2)) - 4*p_2/(m_1*m_2)**2*(np.dot(p_dot_1, p_2) + np.dot(p_1, p_dot_2)) + 2*p_dot_1/m_1**2*(1 + np.dot(p_2, p_2)/m_2**2) + 4*p_1*np.dot(p_2, p_dot_2)/(m_1*m_2)**2
    dc_1dp_2_dot = -4*p_dot_1/(m_1*m_2)*(1 + np.dot(p_1, p_2)/(m_1*m_2)) - 4*p_1/(m_1*m_2)**2*(np.dot(p_dot_1, p_2) + np.dot(p_1, p_dot_2)) + 2*p_dot_2/m_2**2*(1 + np.dot(p_1, p_1)/m_1**2) + 4*p_2*np.dot(p_1, p_dot_1)/(m_1*m_2)**2
    E_dot_1 = np.dot(p_1, p_dot_1)*c**2/E_1
    E_dot_2 = np.dot(p_2, p_dot_2)*c**2/E_2
    q_dot_1 = p_1/E_1 + G*m_1*m_2/r*( -(p_1/E_1*E_2)/(E_1*E_2)**2*c_1 + dc_1dp_1/(E_1*E_2) )
    q_dot_2 = p_2/E_2 + G*m_1*m_2/r*( -(p_2/E_2*E_1)/(E_1*E_2)**2*c_1 + dc_1dp_2/(E_1*E_2) )
    v_1 = q_dot_1
    v_2 = q_dot_2

    r_dot = (r_vec[0]*(v_1[0] - v_2[0]) + r_vec[1]*(v_1[1] - v_2[1]))/r

    # V2PM_p_1 = -G**2/(r*E_1*E_2)**2*( ( 3/4*(m_1**2*m_2**2 - 5*fourdot(P_1, P_2)**2)*(m_1 + m_2)/c**2 + c_1*(E_1 + E_2)/(E_1*E_2)*(c_1*(ξ-1)/(2*(E_1*E_2)) - 4*fourdot(P_1, P_2)/c**2) )*E_2*p_1/E_1*c**2 + E_1*E_2*( 30/4*fourdot(P_1, P_2)*(E_2/E_1*p_1 - p_2)*(m_1 + m_2)/c**2 + ( 4*fourdot(P_1, P_2)*(E_2/E_1*p_1 - p_2)*(E_1 + E_2)/(E_1*E_2) - c_1*p_1/(E_1*E_2*E_1)*c**2 + c_1*(E_1 + E_2)/(E_1*E_2)**2*p_1*c**2 )*(c_1*(ξ-1)/(2*(E_1*E_2)) - 4*fourdot(P_1, P_2)/c**2) + c_1*(E_1 + E_2)(E_1*E_2)*( (4*fourdot(P_1, P_2)*(E_2/E_1*p_1 - p_2)*(ξ - 1) - c_1*p_1/(E_1*(E_1 + E_2)**3)*c**2*(E_2*(E_1 + E_2) - 2*E_1*E_2) )/(2*E_1*E_2) + c_1*(ξ-1)/(E_1*E_2)**2*E_2/E_1*p_1*c**2 + 4*((E_2/E_1*p_1) - p_2)/c**2 ) ) ) 
    

    a_1 = p_dot_1/E_1 - p_1/E_1**2*E_dot_1 - G*m_1*m_2/(E_1**2*E_2**2*r)*( r_dot/r*( E_2/E_1*p_1*E_1*E_2 + E_1*E_2*dc_1dp_1 ) + c_1*( E_dot_2*p_1/E_1 + E_2*( p_dot_1/E_1 - p_1/E_1**2*E_dot_1 ) ) + p_1/E_1*E_2*( c_1_dot - 4*c_1*( E_dot_1*E_2 + E_1*E_dot_2 ) ) - dc_1dp_1_dot*E_1*E_2 + ( E_dot_1*E_2 + E_1*E_dot_2 )*dc_1dp_1 )
    a_2 = p_dot_2/E_2 - p_2/E_2**2*E_dot_2 - G*m_1*m_2/(E_1**2*E_2**2*r)*( r_dot/r*( E_1/E_2*p_2*E_1*E_2 + E_1*E_2*dc_1dp_2 ) + c_1*( E_dot_1*p_2/E_2 + E_1*( p_dot_2/E_2 - p_2/E_2**2*E_dot_2 ) ) + p_2/E_2*E_1*( c_1_dot - 4*c_1*( E_dot_1*E_2 + E_1*E_dot_2 ) ) - dc_1dp_2_dot*E_1*E_2 + ( E_dot_1*E_2 + E_1*E_dot_2 )*dc_1dp_2 )
    return np.array([v_1, v_2, a_1, a_2])

def boost(v_1, v_2, a_1, a_2, dt):
    v_1 += a_1*dt
    v_2 += a_2*dt
    return np.array([v_1, v_2])

def move(x_1, x_2, v_1, v_2, dt):
    x_1 += v_1*dt
    x_2 += v_2*dt
    return np.array([x_1, x_2])

# def punch(v_1, v_2, m_1, m_2, DL):
#     p_1 = γ(v_1, DL)*v_1*m_1
#     p_2 = γ(v_2, DL)*v_2*m_2
#     return np.array([p_1, p_2])

In [ ]:
def run_model(s, t_max, dt, DL = False, α = 1, return_momenta = False):
    if DL == True:
        G, c = 1, 1
    else:
        c = const.c.value
        G = const.G.value
    
    x_1, x_2, v_1, v_2, m_1, m_2 = s
    a_1, a_2 = 0, 0
    
    P_1, P_2 = punch(v_1, v_2, m_1, m_2, DL)
    R_S = findR_S(P_1, P_2, DL)

    pos1 = np.zeros((2, int(t_max/dt)))
    pos2 = np.zeros((2, int(t_max/dt)))
    pos_CM = np.zeros((2, int(t_max/dt)))
    mom1 = np.zeros((3, int(t_max/dt)))
    mom2 = np.zeros((3, int(t_max/dt)))

    r_init = rel_len(x_1, x_2)

    for i in range(int(t_max/dt)):
        if rel_len(x_1, x_2) <= 5*R_S:
            print('Code interrupted: Radial Dip')
            pos1, pos2, pos_CM = pos1[:,0:i], pos2[:,0:i], pos_CM[:,0:i]
            mom1, mom2 = mom1[:,0:i], mom2[:,0:i]
            break
        elif rel_len(x_1, x_2) >= 5*r_init:
            print('Code interrupted: Max distance')
            pos1, pos2, pos_CM = pos1[:,0:i], pos2[:,0:i], pos_CM[:,0:i]
            mom1, mom2 = mom1[:,0:i], mom2[:,0:i]
            break

        pos1[:,i] = x_1
        pos2[:,i] = x_2
        pos_CM[:,i] = findrCM(x_1, x_2, m_1, m_2)

        P_1, P_2 = punch(v_1, v_2, m_1, m_2, DL)
        mom1[:,i] = P_1
        mom2[:,i] = P_2

        v_1, v_2, a_1, a_2 = accelerationPM(x_1, x_2, P_1, P_2, m_1, m_2, DL, α)
        v_1, v_2 = boost(v_1, v_2, a_1, a_2, dt)
        x_1, x_2 = move(x_1, x_2, v_1, v_2, dt)

        # r = rel_len(x_1,x_2)
        # print(r, x_1, x_2, v_1, v_2, a_1, a_2)

    momenta = np.array([mom1, mom2])
    positions = np.array([pos1, pos2, pos_CM])
    if return_momenta == True:
        return positions, momenta
    else:
        return positions


def run_modelN(s, t_max, dt, DL = False, α = 1, return_momenta=False):
    if DL == True:
        G, c = 1, 1
    else:
        c = const.c.value
        G = const.G.value
    
    x_1, x_2, v_1, v_2, m_1, m_2 = s
    a_1, a_2 = 0, 0
    
    R_S = findR_SN(m_1, m_2, DL)

    pos1N = np.zeros((2, int(t_max/dt)))
    pos2N = np.zeros((2, int(t_max/dt)))
    pos_CMN = np.zeros((2, int(t_max/dt)))
    mom1N = np.zeros((2, int(t_max/dt)))
    mom2N = np.zeros((2, int(t_max/dt)))

    r_init = rel_len(x_1, x_2)

    for i in range(int(t_max/dt)):
        if rel_len(x_1, x_2) <= R_S:
            print('Code interrupted: Radial Dip')
            pos1N, pos2N, pos_CMN = pos1N[:,0:i], pos2N[:,0:i], pos_CMN[:,0:i]
            break
        elif rel_len(x_1, x_2) >= 5*r_init:
            print('Code interrupted: Max distance')
            pos1N, pos2N, pos_CMN = pos1N[:,0:i], pos2N[:,0:i], pos_CMN[:,0:i]
            break

        pos1N[:,i] = x_1
        pos2N[:,i] = x_2
        pos_CMN[:,i] = findrCM(x_1, x_2, m_1, m_2)
        mom1N[:,i] = np.dot(v_1,v_1)*m_1/2
        mom2N[:,i] = np.dot(v_2,v_2)*m_2/2

        a_1, a_2 = accelerationN(x_1, x_2, m_1, m_2, DL)
        v_1, v_2 = boost(v_1, v_2, a_1, a_2, dt)
        x_1, x_2 = move(x_1, x_2, v_1, v_2, dt)

    positionsN = np.array([pos1N, pos2N, pos_CMN])
    momentaN = np.array([mom1N, mom2N])
    if return_momenta == True:
        return positionsN, momentaN
    else:
        return positionsN

In [ ]:
def findMaxPos(pos):
    xmax, xmin, ymax, ymin = max(pos[0]), min(pos[0]), max(pos[1]), min(pos[1])
    return xmax, xmin, ymax, ymin

def plotLims(pos):
    start_pos = np.array([pos[0][0], pos[1][0]])
    end_pos = np.array([pos[0][-1], pos[1][-1]])

    xmax, xmin, ymax, ymin = findMaxPos(pos)
    max_pos = max(max(abs(start_pos)),max(abs(end_pos)))

    xlim = (xmin-0.1*max_pos, xmax+0.1*max_pos)
    ylim = (ymin-0.1*max_pos, ymax+0.1*max_pos)
    return xlim, ylim

def plotLimsTwoBody(pos1, pos2):
    start_pos1 = np.array([pos1[0][0], pos1[1][0]])
    end_pos1 = np.array([pos1[0][-1], pos1[1][-1]])
    start_pos2 = np.array([pos2[0][0], pos2[1][0]])
    end_pos2 = np.array([pos2[0][-1], pos2[1][-1]])

    x1max, x1min, y1max, y1min = findMaxPos(pos1)
    x2max, x2min, y2max, y2min = findMaxPos(pos2)

    xmax = max(x1max, x2max)
    xmin = min(x1min, x2min)
    ymax = max(y1max, y2max)
    ymin = min(y1min, y2min)

    max_pos1 = max(max(abs(start_pos1)),max(abs(end_pos1)))
    max_pos2 = max(max(abs(start_pos2)),max(abs(end_pos2)))
    max_pos = max(max_pos1, max_pos2)

    xlim = (xmin-0.1*max_pos, xmax+0.1*max_pos)
    ylim = (ymin-0.1*max_pos, ymax+0.1*max_pos)
    return xlim, ylim

def orbPlotter(positions, positionsN = 0, xlim = 0, ylim = 0, slice = slice(0, -1, 1), aspect = 1, filename='', CM = True, save = False, show = True, DL = False, N = False, figsize=(8,8)):
    if DL == False:
        x_1, x_2, x_cm = positions[0:3,:,slice]/149597871000
    else:
        x_1, x_2, x_cm = positions[0:3,:,slice]
    if CM == True:
        x_1 = x_1 - x_cm
        x_2 = x_2 - x_cm
        x_cm = np.zeros_like(x_1)

    fig, ax = plt.subplots(figsize=figsize)
    if type(xlim) != tuple:
        xlim = plotLimsTwoBody(x_1, x_2)[0]
    if type(ylim) != tuple:
        ylim = plotLimsTwoBody(x_1, x_2)[1]
    
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    if type(positionsN) == np.ndarray:
        if DL == False:
            x_1N, x_2N, x_cmN = positionsN[0:3,:,slice]/149597871000
        else:
            x_1N, x_2N, x_cmN = positionsN[0:3,:,slice]
        if CM == True:
            x_1N = x_1N - x_cmN
            x_2N = x_2N - x_cmN
        ax.plot(x_1N[0], x_1N[1], 'y', lw=0.5, label='Newtonian motion')
        ax.plot(x_2N[0], x_2N[1], 'y', lw=0.5, label='Newtonian motion')

    if CM == False:
        ax.plot(x_cm[0], x_cm[1], 'g:',label = 'CM')
        ax.plot(x_cm[0][0], x_cm[1][0], 'g.',markersize=8)
        ax.plot(x_cm[0][-1], x_cm[1][-1], 'g.',markersize=8)

    ax.plot(x_1[0], x_1[1],'b')
    ax.plot(x_1[0][0], x_1[1][0], 'b.', label = 'm_1 start', markersize=15)
    ax.plot(x_1[0][-1], x_1[1][-1], 'bo', label = 'm_1 stop',markersize=15)

    ax.plot(x_2[0], x_2[1],'r')
    ax.plot(x_2[0][0], x_2[1][0], 'r.', label = 'm_2 start', markersize=15)
    ax.plot(x_2[0][-1], x_2[1][-1], 'ro', label = 'm_2 stop',markersize=15)

    if DL == True:
        ax.set_xlabel('$R_S$', fontsize = 15)
        ax.set_ylabel('$R_S$', fontsize = 15)
    else:
        ax.set_xlabel('$x \ [\mathrm{AU}]$', fontsize = 15)
        ax.set_ylabel('$y \ [\mathrm{AU}]$', fontsize = 15)  
    ax.grid(c='grey', alpha=0.2, ls ='--')
    ax.set_aspect(aspect)
    ax.set_title(f'{filename}', fontsize = 20)
    ax.legend(facecolor='grey', fontsize = 12, bbox_to_anchor=(1.01,1), loc='upper left')

    # d = {
    #      'E:':      parameters[0],
    #      '|p_1|:':  parameters[1],
    #      '|p_2|:':  parameters[2],
    #      '|L_1|:':  parameters[3],
    #      '|L_2|:':  parameters[4],
    #      'R_S:':    parameters[5],
    #      'b:':      parameters[6],
    #      'θ:':      parameters[7],
    #     }

    # text = nice_string_output(d, extra_spacing=3, decimals=2)
    # add_text_to_ax(1.04, 0, text, ax, fontsize=12)

    plt.title(f'{filename}', fontsize=25, y=1.08)
    fig.patch.set_facecolor('white')
    fig.tight_layout()
    if show == False:
        plt.close(fig)
    if save == True:
        fig.savefig(f'./Plots/{filename}.png', dpi=300, transparent = False)
    return

In [ ]:
# def orbPlotterOLD(positions, positionsN = 0, slice = slice(1, -1, 1), aspect = 1, filename='', CM = True, save = False, show = True, DL = False, figsize=(8,8)):
#     x_1, x_2, x_cm = positions[0:3,:,slice]
#     if CM == True:
#         x_1 = x_1 - x_cm
#         x_2 = x_2 - x_cm
#         x_cm = np.zeros_like(x_1)

#     fig, ax = plt.subplots(figsize=figsize)
#     xlim, ylim = plotLimsTwoBody(x_1, x_2)
#     ax.set_xlim(xlim)
#     ax.set_ylim(ylim)

#     # ax.plot(x_cm[0], x_cm[1], 'g:', label = 'CM', lw=0.4)
#     ax.plot(x_cm[0], x_cm[1], 'g:', lw=0.5)
#     ax.plot(x_cm[0][0], x_cm[1][0], 'g.',markersize=8)
#     ax.plot(x_cm[0][-1], x_cm[1][-1], 'g.',markersize=8)

#     # ax.plot(x_1[0], x_1[1],'b', label = 'm_1', lw=0.75)
#     ax.plot(x_1[0], x_1[1],'b', lw=1)
#     ax.plot(x_1[0][0], x_1[1][0], 'b.', label = 'm_1 start', markersize=15)
#     ax.plot(x_1[0][-1], x_1[1][-1], 'bo', label = 'm_1 stop',markersize=15)

#     # ax.plot(x_2[0], x_2[1],'r', label = 'm_2', lw=0.75)
#     ax.plot(x_2[0], x_2[1],'r', lw=1)
#     ax.plot(x_2[0][0], x_2[1][0], 'r.', label = 'm_2 start', markersize=15)
#     ax.plot(x_2[0][-1], x_2[1][-1], 'ro', label = 'm_2 stop',markersize=15)

#     if type(positionsN) == np.ndarray:
#         x_1N, x_2N, x_cmN = positionsN[0:3,:,slice]
#         if CM == True:
#             x_1N = x_1N - x_cmN
#             x_2N = x_2N - x_cmN
#             x_cmN = np.zeros_like(x_1N)
#         ax.plot(x_1N[0], x_1N[1], 'y:', label='Newtonian motion')

#     if DL == True:
#         ax.set_xlabel('$R_S$', fontsize = 15)
#         ax.set_ylabel('$R_S$', fontsize = 15)
#     else:
#         ax.set_xlabel('$x \ [\mathrm{m}]$', fontsize = 15)
#         ax.set_ylabel('$y \ [\mathrm{m}]$', fontsize = 15)  
#     ax.grid(c='grey', alpha=0.2, ls ='--')
#     ax.set_aspect(aspect)
#     ax.set_title(f'{filename}', fontsize = 20)
#     ax.legend(facecolor='grey', fontsize = 12, bbox_to_anchor=(1.01,1), loc='upper left')

#     # d = {
#     #      'E:':      parameters[0],
#     #      '|p_1|:':  parameters[1],
#     #      '|p_2|:':  parameters[2],
#     #      '|L_1|:':  parameters[3],
#     #      '|L_2|:':  parameters[4],
#     #      'R_S:':    parameters[5],
#     #      'b:':      parameters[6],
#     #      'θ:':      parameters[7],
#     #     }

#     # text = nice_string_output(d, extra_spacing=3, decimals=2)
#     # add_text_to_ax(1.04, 0, text, ax, fontsize=12)

#     plt.title(f'{filename}', fontsize=25, y=1.08)
#     fig.patch.set_facecolor('white')
#     fig.tight_layout()
#     if show == False:
#         plt.close(fig)
#     if save == True:
#         fig.savefig(f'./Plots/{filename}.png', dpi=300, transparent = False)
#     return